In [1]:
#############################################################
# 1. Libraries

import pandas as pd
import numpy as np 
import os
import glob
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split
import tensorflow as tf
tf.keras.backend.clear_session()

physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

from tensorflow.keras import models, layers, regularizers, metrics, losses, optimizers
from tensorflow.keras.utils import Sequence

import matplotlib.pyplot as plt
import gc

import xgboost as xgb
import pickle
import librosa
import scipy.signal
import albumentations

#############################################################

In [2]:
#############################################################
# 2. Paths & Global Variables

## 2.1 Paths

path = '../../../01_Data/'
path_sequences = path + '01_GeneratedSequences/'
path_spectograms_tiny = path + '02_GeneratedSpectograms_MelFeatures/'
path_spectograms_big = path + '03_GeneratedSpectograms_Scipy/'
path_spectograms_stft = path + '04_GeneratedSpectogramsSTFT/'

path_models_mha = [f'../models/model_mha_{num_fold}' for num_fold in range(5)]
path_models_spectogram_tiny = [f'../models/model_cnn2d_Tiny_{num_fold}' for num_fold in range(5)]
path_models_spectogram_big = [f'../models/model_cnn2d_Big_{num_fold}' for num_fold in range(5)]
path_models_spectogram_stft = [f'../models/model_cnn2d_STFT_{num_fold}' for num_fold in range(5)]
path_models_lgbm_stft = [f'../models/model_lgbm_STFT_{num_fold}' for num_fold in range(5)]

df_train = pd.read_csv(path + 'train.csv')
df_sample_submission = pd.read_csv(path + 'sample_submission.csv') 

train_paths = glob.glob(path + 'train/*')
test_paths = glob.glob(path + 'test/*')

unique_segments_id_train = set(df_train['segment_id'])
unique_segments_id_test = set(df_sample_submission['segment_id'])

dict_unique_segments_id = { v : k for k, v in enumerate(unique_segments_id_train)}
dict_unique_segments_id_inv = { k : v for k, v in enumerate(unique_segments_id_train)}

## 2.2 Global Variables

SEQ_LENGTH = 60_001

IMG_SIZE_TINY = (40, 118)
IMG_SIZE_BIG = (128, 235)
IMG_SIZE_STFT = (128, 469)

# SAMPLING_RATE = 100 
# DURATION = 600
# SAMPLES = SAMPLING_RATE * DURATION
# N_MELS = 128
# HOP_LENGTH = 256 # 256
# FMIN = 0
# FMAX = SAMPLING_RATE // 2
# N_FFT = N_MELS * 20 #2048

# N_MFCC = 40
# MAX_PAD_LENGTH = 118

#############################################################

In [3]:
#############################################################
# 3. Global Functions

def buildSequences(df, dict_segment_paths, training=True, mask_value=-1.0):
    x = np.zeros((len(dict_segment_paths), SEQ_LENGTH, 10))
    if training:
        y = np.zeros(len(dict_segment_paths))
    for i, segment in enumerate(tqdm(dict_segment_paths, total=len(dict_segment_paths), position=0)):
        segment_path = dict_segment_paths[segment]
        df_tmp = pd.read_csv(segment_path)
        df_tmp = df_tmp.fillna(mask_value)
        x[i] = df_tmp.values[-SEQ_LENGTH:]
        if training:
            y[i] = df['time_to_eruption'][df['segment_id']==segment].values[0]
    if training:
        return x, y
    else:
        return x


def scale(x, mean_, std_):
    return (x - mean_) / std_


def unscale(x, mean_, std_):
    return (x * std_) + mean_


def monoToColor(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
#     X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean) / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Scale to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V


# # 5.2 Augmentations

# def noiseInjection(batch_sequences, noise_factor=0.075):
#     noise = np.random.randn(batch_sequences.shape[0], batch_sequences.shape[1], batch_sequences.shape[2])
#     augmented_data = batch_sequences + noise_factor * noise
#     return augmented_data


# def timeShifting(batch_sequences, shift_max):
#     shift = np.random.randint(shift_max)
#     for sensor in range(10):
#         batch_sequences[:, :, sensor] = np.roll(batch_sequences[:, :, sensor], shift)
#     return batch_sequences

# def makeAugmentations(list_segments, dict_path_data, seq_length):
    
#     batch_sequences = np.asarray([np.load(dict_path_data[segment]) for segment in list_segments])
    
#     list_augmentations = [0, 1, 2, 3]
#     current_augmentations = list(np.random.choice(list_augmentations, size=np.random.randint(1, 4) ,replace=False))
      
#     # Add random noise
#     if 0 in current_augmentations:
#         batch_sequences = noiseInjection(batch_sequences, noise_factor=0.05)   

#     # Time shifting
#     if 1 in current_augmentations:
#         batch_sequences = timeShifting(batch_sequences, shift_max=6_000) 

#     # Random batch sequence sensors slices to null
#     if 2 in current_augmentations:
#         num_random_sensors = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
#         random_sensors = list(set(np.random.randint(0, 9, size=num_random_sensors)))
#         random_ini_position = np.random.randint(0, seq_length, size=num_random_sensors)
#         random_length = np.random.randint(random_ini_position, random_ini_position+6_000, size=num_random_sensors)
#         random_length -= random_ini_position

#         if num_random_sensors!=0:
#             for i, sensor in enumerate(random_sensors):
#                 batch_sequences[:, random_ini_position[i]:random_ini_position[i]+random_length[i], sensor] = 0.0

#     # Shut-down sensor
#     if 3 in current_augmentations:
#         sensor = np.random.randint(0, 9)
#         batch_sequences[:, :, sensor] = 0.0
        
#     return batch_sequences


#############################################################

In [4]:
#############################################################
# 4. Preprocess

# Mha
dict_segments_sequences_paths_train = {
    segment : path_sequences + 'train/' + str(segment) + '.npy' for segment in unique_segments_id_train
}

dict_segments_sequences_paths_test = {
    segment : path_sequences + 'test/' + str(segment) + '.npy' for segment in unique_segments_id_test
}

# Spectogram - Tiny
dict_segments_spectograms_tiny_paths_train = {
    segment : path_spectograms_tiny + 'train/' + str(segment) + '/' for segment in unique_segments_id_train
}

dict_segments_spectograms_tiny_paths_test = {
    segment : path_spectograms_tiny + 'test/' + str(segment) + '/' for segment in unique_segments_id_test
}

# Spectogram - Big

dict_segments_spectograms_big_paths_train = {
    segment : path_spectograms_big + 'train/' + str(segment) + '/' for segment in unique_segments_id_train
}

dict_segments_spectograms_big_paths_test = {
    segment : path_spectograms_big + 'test/' + str(segment) + '/' for segment in unique_segments_id_test
}

# Spectogram - STFT

dict_segments_spectograms_stft_paths_train = {
    segment : path_spectograms_stft + 'train/' + str(segment) + '/' for segment in unique_segments_id_train
}

dict_segments_spectograms_stft_paths_test = {
    segment : path_spectograms_stft + 'test/' + str(segment) + '/' for segment in unique_segments_id_test
}

######

dict_positions_segments = {k : i for i, k in enumerate(dict_segments_sequences_paths_train.keys())}

df_train['time_to_eruption'] = df_train['time_to_eruption']/(10**6)

dict_labels = {
    segment : df_train['time_to_eruption'][df_train['segment_id']==segment].values.flatten()
    for segment in unique_segments_id_train
}

###

# dict_nans_train = getdDictsSpectoGramsNulls(dict_segment_paths_train)
# dict_nans_test = getdDictsSpectoGramsNulls(dict_segment_paths_test)

# np.save(path + 'dict_nans_train.npy', dict_nans_train)
# np.save(path + 'dict_nans_test.npy', dict_nans_test)

dict_nans_train = np.load(path + 'dict_nans_train.npy', allow_pickle=True).flatten()[0]
dict_nans_test = np.load(path + 'dict_nans_test.npy', allow_pickle=True).flatten()[0]


#############################################################

In [5]:
#############################################################
# 5. Global Functions

def getTinyTransforms():
    return albumentations.Compose([
            albumentations.OneOf([
                albumentations.GaussNoise(p=0.2),
                albumentations.Cutout(num_holes=4, max_h_size=6, max_w_size=6, fill_value=0, p=0.2),
            ], p=0.3),
    ])

def getBigTransforms():
    return albumentations.Compose([
            albumentations.OneOf([
                albumentations.GaussNoise(p=0.2),
                albumentations.Cutout(num_holes=8, max_h_size=12, max_w_size=12, fill_value=0, p=0.2),
            ], p=0.3),
        albumentations.OpticalDistortion(p=0.3),
        albumentations.ShiftScaleRotate(shift_limit=0.05, rotate_limit=1, p=0.5),
        albumentations.RandomCrop(IMG_SIZE_BIG[0]-10, IMG_SIZE_BIG[1]-10, p=0.5),
        albumentations.PadIfNeeded(min_height=IMG_SIZE_BIG[0], min_width=IMG_SIZE_BIG[1], value=0, p=1.0)
    ])

def getStftTransforms():
    return albumentations.Compose([
            albumentations.OneOf([
                albumentations.GaussNoise(p=0.2),
                albumentations.Cutout(num_holes=8, max_h_size=12, max_w_size=12, fill_value=0, p=0.2),
            ], p=0.3),
        albumentations.OpticalDistortion(p=0.3),
        albumentations.ShiftScaleRotate(shift_limit=0.05, rotate_limit=1, p=0.5),
        albumentations.RandomCrop(IMG_SIZE_STFT[0]-10, IMG_SIZE_STFT[1]-10, p=0.5),
        albumentations.PadIfNeeded(min_height=IMG_SIZE_STFT[0], min_width=IMG_SIZE_STFT[1], value=0, p=1.0)
    ])

#############################################################

In [6]:
#############################################################
# 6. Generator

# 5.1 MHA Model Data Generator
class MHAVolcanoSequencesGenerator(Sequence):
    
    def __init__(self, segments, path_sequences, batch_size, dict_labels, augmentations, shuffle=False, training=True):
        super(MHAVolcanoSequencesGenerator, self).__init__()
        
        self.dict_means = {0: 0.09421943291597953, 1: 0.9208114415834104, 2: -0.026617075839858038, 
                           3: 0.09724443370400684, 4: 1.704695380910225, 5: -0.1180321202370159, 6: 0.7667902421713446, 
                           7: 0.7804286101804458, 8: -0.2075797991904395, 9: 0.014516944212624944} 
        
        self.dict_stds =  {0: 1820.6211174856987, 1: 1931.0901612736805, 2: 1738.1671740163413, 
                           3: 1669.8837574619292, 4: 568.5221048211192, 5: 1848.4917466767877, 6: 1623.353060255481, 
                           7: 1618.2714709240895, 8: 1590.9403316558762, 9: 1906.41447528788}
        
        self.segments = segments
        self.path_sequences = path_sequences
        self.batch_size = batch_size
        self.dict_labels = dict_labels
        self.augmentations = augmentations
        self.shuffle = shuffle
        self.training = training
        self.on_epoch_end()
        
    def __len__(self):
        self.num_steps = int(np.ceil(len(self.segments) / self.batch_size))
        return self.num_steps
        
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        list_batch_segments = [self.segments[k] for k in indexes]
        
        
        array_sequences = np.asarray([np.load(self.path_sequences[segment], allow_pickle=True)[-SEQ_LENGTH:, :]
                                     for segment in list_batch_segments])
        
        if self.augmentations:
            array_sequences = self.augmentBatch(array_sequences)
        
        array_sequences[:, :, 0] = scale(array_sequences[:, :, 0], self.dict_means[0], self.dict_stds[0])
        array_sequences[:, :, 1] = scale(array_sequences[:, :, 1], self.dict_means[1], self.dict_stds[1])
        array_sequences[:, :, 2] = scale(array_sequences[:, :, 2], self.dict_means[2], self.dict_stds[2])
        array_sequences[:, :, 3] = scale(array_sequences[:, :, 3], self.dict_means[3], self.dict_stds[3])
        array_sequences[:, :, 4] = scale(array_sequences[:, :, 4], self.dict_means[4], self.dict_stds[4])
        array_sequences[:, :, 5] = scale(array_sequences[:, :, 5], self.dict_means[5], self.dict_stds[5])
        array_sequences[:, :, 6] = scale(array_sequences[:, :, 6], self.dict_means[6], self.dict_stds[6])
        array_sequences[:, :, 7] = scale(array_sequences[:, :, 7], self.dict_means[7], self.dict_stds[7])
        array_sequences[:, :, 8] = scale(array_sequences[:, :, 8], self.dict_means[8], self.dict_stds[8])
        array_sequences[:, :, 9] = scale(array_sequences[:, :, 9], self.dict_means[9], self.dict_stds[9])
        
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_batch_segments])
            return array_sequences, array_labels
        else:
            return array_sequences
        
        
    def noiseInjection(self, batch_sequences, noise_factor=0.075):
        noise = np.random.randn(batch_sequences.shape[0], batch_sequences.shape[1], batch_sequences.shape[2])
        augmented_data = batch_sequences + noise_factor * noise
        return augmented_data
    
    
    def timeShifting(self, batch_sequences, shift_max):
        shift = np.random.randint(shift_max)
        for sensor in range(10):
            batch_sequences[:, :, sensor] = np.roll(batch_sequences[:, :, sensor], shift)
        return batch_sequences
       
    
    def augmentBatch(self, batch_sequences):
        
        # Add random noise
        if np.random.random() > 0.5:
            batch_sequences = self.noiseInjection(batch_sequences, noise_factor=0.005)   
            
        # Time shifting
        if np.random.random() > 0.5:
            batch_sequences = self.timeShifting(batch_sequences, shift_max=600) 
                           
        # Shut-down sensor
        if np.random.random() > 0.5:
            sensor = np.random.randint(0, 9)
            batch_sequences[:, :, sensor] = 0.0
                
        return batch_sequences
    
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.segments))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        
        

# 5.2 Spectogram Model Data Generator
class SpectoGramVolcanoSequencesGenerator(Sequence):
    
    def __init__(self, segments, path_spectograms, batch_size, dict_labels, transforms, shuffle=False, training=True):
        super(SpectoGramVolcanoSequencesGenerator, self).__init__()
        self.segments = segments
        self.path_spectograms = path_spectograms
        self.batch_size = batch_size
        self.dict_labels = dict_labels
        self.shuffle = shuffle
        self.training = training
        self.transforms = transforms
        self.on_epoch_end()
        
    def __len__(self):
        self.num_steps = int(np.ceil(len(self.segments) / self.batch_size))
        return self.num_steps
        
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        list_batch_segments = [self.segments[k] for k in indexes]
        
        array_spectograms_s0 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_0.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s1 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_1.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s2 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_2.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s3 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_3.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s4 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_4.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s5 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_5.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s6 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_6.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s7 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_7.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s8 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_8.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s9 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_9.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        
        if self.transforms:
            data_s0, data_s1 = {'image':array_spectograms_s0}, {'image':array_spectograms_s1}
            data_s2, data_s3 = {'image':array_spectograms_s2}, {'image':array_spectograms_s3}
            data_s4, data_s5 = {'image':array_spectograms_s4}, {'image':array_spectograms_s5}
            data_s6, data_s7 = {'image':array_spectograms_s6}, {'image':array_spectograms_s7}
            data_s8, data_s9 = {'image':array_spectograms_s8}, {'image':array_spectograms_s9}
            
            array_spectograms_s0 = np.stack([self.transforms(image=x)['image'] for x in data_s0['image']], axis=0)
            array_spectograms_s1 = np.stack([self.transforms(image=x)['image'] for x in data_s1['image']], axis=0)
            array_spectograms_s2 = np.stack([self.transforms(image=x)['image'] for x in data_s2['image']], axis=0)
            array_spectograms_s3 = np.stack([self.transforms(image=x)['image'] for x in data_s3['image']], axis=0)
            array_spectograms_s4 = np.stack([self.transforms(image=x)['image'] for x in data_s4['image']], axis=0)
            array_spectograms_s5 = np.stack([self.transforms(image=x)['image'] for x in data_s5['image']], axis=0)
            array_spectograms_s6 = np.stack([self.transforms(image=x)['image'] for x in data_s6['image']], axis=0)
            array_spectograms_s7 = np.stack([self.transforms(image=x)['image'] for x in data_s7['image']], axis=0)
            array_spectograms_s8 = np.stack([self.transforms(image=x)['image'] for x in data_s8['image']], axis=0)
            array_spectograms_s9 = np.stack([self.transforms(image=x)['image'] for x in data_s9['image']], axis=0)
                 
        batch = (array_spectograms_s0/255, array_spectograms_s1/255, array_spectograms_s2/255, array_spectograms_s3/255, 
                 array_spectograms_s4/255, array_spectograms_s5/255, array_spectograms_s6/255, array_spectograms_s7/255, 
                 array_spectograms_s8/255, array_spectograms_s9/255)    
            
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_batch_segments])
        if self.training:
            return batch, array_labels
        else:
            return batch
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.segments))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        
    
        
        
# 5.3 STFT Model Data Generator
class STFTVolcanoSequencesGenerator(Sequence):
    
    def __init__(self, segments, path_spectograms, batch_size, dict_labels, transforms, shuffle=False, training=True):
        super(STFTVolcanoSequencesGenerator, self).__init__()
        
        self.segments = segments
        self.path_spectograms = path_spectograms
        self.batch_size = batch_size
        self.dict_labels = dict_labels
        self.transforms = transforms
        self.shuffle = shuffle
        self.training = training
        self.on_epoch_end()
        
    def __len__(self):
        self.num_steps = int(np.ceil(len(self.segments) / self.batch_size))
        return self.num_steps
        
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        list_batch_segments = [self.segments[k] for k in indexes]
        
        array_spectograms_s0 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_0.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s1 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_1.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s2 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_2.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s3 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_3.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s4 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_4.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s5 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_5.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s6 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_6.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s7 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_7.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s8 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_8.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s9 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_9.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8)  
        
        if self.transforms:
            data_s0, data_s1 = {'image':array_spectograms_s0}, {'image':array_spectograms_s1}
            data_s2, data_s3 = {'image':array_spectograms_s2}, {'image':array_spectograms_s3}
            data_s4, data_s5 = {'image':array_spectograms_s4}, {'image':array_spectograms_s5}
            data_s6, data_s7 = {'image':array_spectograms_s6}, {'image':array_spectograms_s7}
            data_s8, data_s9 = {'image':array_spectograms_s8}, {'image':array_spectograms_s9}
            
            array_spectograms_s0 = np.stack([self.transforms(image=x)['image'] for x in data_s0['image']], axis=0)
            array_spectograms_s1 = np.stack([self.transforms(image=x)['image'] for x in data_s1['image']], axis=0)
            array_spectograms_s2 = np.stack([self.transforms(image=x)['image'] for x in data_s2['image']], axis=0)
            array_spectograms_s3 = np.stack([self.transforms(image=x)['image'] for x in data_s3['image']], axis=0)
            array_spectograms_s4 = np.stack([self.transforms(image=x)['image'] for x in data_s4['image']], axis=0)
            array_spectograms_s5 = np.stack([self.transforms(image=x)['image'] for x in data_s5['image']], axis=0)
            array_spectograms_s6 = np.stack([self.transforms(image=x)['image'] for x in data_s6['image']], axis=0)
            array_spectograms_s7 = np.stack([self.transforms(image=x)['image'] for x in data_s7['image']], axis=0)
            array_spectograms_s8 = np.stack([self.transforms(image=x)['image'] for x in data_s8['image']], axis=0)
            array_spectograms_s9 = np.stack([self.transforms(image=x)['image'] for x in data_s9['image']], axis=0)
                 
        batch = (array_spectograms_s0/255, array_spectograms_s1/255, array_spectograms_s2/255, array_spectograms_s3/255, 
                 array_spectograms_s4/255, array_spectograms_s5/255, array_spectograms_s6/255, array_spectograms_s7/255, 
                 array_spectograms_s8/255, array_spectograms_s9/255)       
            
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_batch_segments])
            return batch, array_labels
        else:
            return batch
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.segments))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        

# 5.4
def buildDataset(dict_paths):
            
    fs = 100                # sampling frequency 
    n = 256                 # FFT segment size
    max_f = 20              # ～20Hz

    delta_f = fs / n        # 0.39Hz
    delta_t = n / fs / 2    # 1.28s

    feature_set = []
    for segment_id in tqdm(dict_paths, total=len(dict_paths), position=0):
        data = np.load(dict_paths[segment_id])
        segment = [segment_id]
        for sensor in range(10):
            x = data[:, sensor]
            f, t, Z = scipy.signal.stft(x, fs = fs, window = 'hann', nperseg = n)
            f = f[:round(max_f/delta_f)+1]
            Z = np.abs(Z[:round(max_f/delta_f)+1]).T

            th = Z.mean() * 1 
            Z_pow = Z.copy()
            Z_pow[Z < th] = 0
            Z_num = Z_pow.copy()
            Z_num[Z >= th] = 1

            Z_pow_sum = Z_pow.sum(axis = 0)
            Z_num_sum = Z_num.sum(axis = 0)

            A_pow = Z_pow_sum[round(10/delta_f):].sum()
            A_num = Z_num_sum[round(10/delta_f):].sum()
            BH_pow = Z_pow_sum[round(5/delta_f):round(8/delta_f)].sum()
            BH_num = Z_num_sum[round(5/delta_f):round(8/delta_f)].sum()
            BL_pow = Z_pow_sum[round(1.5/delta_f):round(2.5/delta_f)].sum()
            BL_num = Z_num_sum[round(1.5/delta_f):round(2.5/delta_f)].sum()
            C_pow = Z_pow_sum[round(0.6/delta_f):round(1.2/delta_f)].sum()
            C_num = Z_num_sum[round(0.6/delta_f):round(1.2/delta_f)].sum()
            D_pow = Z_pow_sum[round(2/delta_f):round(4/delta_f)].sum()
            D_num = Z_num_sum[round(2/delta_f):round(4/delta_f)].sum()
            segment += [A_pow, A_num, BH_pow, BH_num, BL_pow, BL_num, C_pow, C_num, D_pow, D_num]

        feature_set.append(segment)

    cols = ['segment_id']
    for i in range(10):
        for j in ['A_pow', 'A_num','BH_pow', 'BH_num','BL_pow', 'BL_num','C_pow', 'C_num','D_pow', 'D_num']:
            cols += [f's{i+1}_{j}']
    feature_df = pd.DataFrame(feature_set, columns = cols)
    feature_df['segment_id'] = feature_df['segment_id'].astype('int')
    
    return feature_df
    
#############################################################

In [7]:
#############################################################
# 6. Build Train Set

batch_size = 8
qt_augmentations = 10

pbar = tqdm(total=(len(path_models_mha)*5), position=0)
list_segments_train = list(unique_segments_id_train)
kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=12)

dict_predictions = {'tiny' : {}, 'big': {}, 'mha': {}, 'lgbm_stft': {}, 'cnn2d_stft': {}, 'y_true': {}, 'segment_id': {}}        
for num_fold, (train_index, val_index) in enumerate(kf.split(list_segments_train,
                                                     np.zeros(len(list_segments_train)))):

    paths = (path_models_mha[num_fold], path_models_spectogram_stft[num_fold],  
             path_models_lgbm_stft[num_fold], path_models_spectogram_tiny[num_fold], 
             path_models_spectogram_big[num_fold])
    
    for path in paths: 
        type_model = '_'.join(path.split('_')[-3:-1]).lower() if path.split('_')[-2].lower() == 'stft' else path.split('_')[-2].lower()
        print(type_model)
        if type_model in {'tiny', 'big', 'mha', 'cnn2d_stft'}:
            model = models.load_model(path, compile=False) 
        
        segments_train_fold = np.asarray(list_segments_train)[train_index]
        segments_val_fold = np.asarray(list_segments_train)[val_index]
        
        if type_model in {'tiny'}:
            X_val_generator = SpectoGramVolcanoSequencesGenerator(segments_val_fold, 
                                                                  dict_segments_spectograms_tiny_paths_train,
                                                                  batch_size=batch_size, dict_labels=dict_labels,
                                                                  transforms=getTinyTransforms(),
                                                                  training=True, shuffle=False)     
        elif type_model in {'big'}:
            X_val_generator = SpectoGramVolcanoSequencesGenerator(segments_val_fold, 
                                                                  dict_segments_spectograms_big_paths_train,
                                                                  batch_size=batch_size, dict_labels=dict_labels, 
                                                                  transforms=getBigTransforms(),
                                                                  training=True, shuffle=False)
            
        elif type_model in {'mha'}:
            X_val_generator = MHAVolcanoSequencesGenerator(segments_val_fold, 
                                                          dict_segments_sequences_paths_train,
                                                          batch_size=batch_size, dict_labels=dict_labels, 
                                                          augmentations=True, training=True, shuffle=False)   
        elif type_model in {'cnn2d_stft'}:
            X_val_generator = STFTVolcanoSequencesGenerator(segments_val_fold, 
                                                            dict_segments_spectograms_stft_paths_train,
                                                            batch_size=batch_size, dict_labels=dict_labels, 
                                                            transforms=getStftTransforms(), training=True, shuffle=False)
        elif type_model in {'lgbm_stft'}:
             pass

        else:
            raise ValueError(f'Incorrect Type model, {type_model}')
              
        y_val_target = np.asarray([dict_labels[segment] for segment in segments_val_fold])

        list_segments = list(segments_val_fold)
        array_predictions = np.zeros((len(list_segments)), dtype=np.float32)
        array_confidence = np.zeros((len(list_segments)), dtype=np.float32)
        
        if type_model not in {'lgbm_stft'}:
            array_predictions_std = np.zeros((len(list_segments)), dtype=np.float32)
            array_confidence_std = np.zeros((len(list_segments)), dtype=np.float32)
            list_predictions, list_conf = [], []
            for i in range(qt_augmentations):
                y_pred_val = model.predict(X_val_generator)
                list_predictions.append(y_pred_val[0])
                list_conf.append(y_pred_val[1][:, 2] - y_pred_val[1][:, 0])

            array_predictions = np.asarray(list_predictions).mean(axis=0).squeeze().astype(np.float32)
            array_confidence = np.asarray(list_conf).mean(axis=0).squeeze().astype(np.float32)
            array_predictions_std = np.asarray(list_predictions).std(axis=0).squeeze().astype(np.float32)
            array_confidence_std = np.asarray(list_conf).std(axis=0).squeeze().astype(np.float32)

            dict_predictions[type_model][num_fold] = {
                'pred_mean': array_predictions, 'pred_std': array_predictions_std,
                'conf_mean' : array_confidence, 'conf_std' : array_confidence_std
            }
            
        else: 
            df_X_val = buildDataset({k: dict_segments_sequences_paths_train[k] for k in segments_val_fold})
            df_X_val = df_X_val.merge(df_train[['segment_id', 'time_to_eruption']], how='inner')
            features = [col for col in df_X_val.columns.tolist() if col not in ['segment_id', 'time_to_eruption']]
            X_val = df_X_val[features]
            y_val = df_X_val['time_to_eruption']
            model = pickle.load(open(f'{path}.pickle', 'rb'))
            array_predictions = model.predict(X_val)
            array_confidence[:] = 1.0
            dict_predictions[type_model][num_fold] = {'pred' : array_predictions, 'conf' : array_confidence}
            
        
        pbar.update(1)
            
        dict_predictions['y_true'][num_fold] = y_val_target
        dict_predictions['segment_id'][num_fold] = segments_val_fold
        
        del model
        gc.collect()
        
pbar.close()

np.save(f'./dict_predictions.npy', dict_predictions) 
dict_predictions = np.load(f'./dict_predictions.npy', allow_pickle=True).flatten()[0]

# del list_predictions
# gc.collect()

columns_names = []
for key in dict_predictions.keys():
    if key in ['mha', 'big', 'tiny', 'cnn2d_stft', 'lgbm_stft']:
        for fold in list(dict_predictions[key].keys()):
            if isinstance(dict_predictions[key][fold], dict):
                for var in dict_predictions[key][fold]:
                    columns_names.append(f'{key}_{fold}_{var}')  
            else:
                columns_names.append(f'{key}_{fold}')
                
columns_names = columns_names + ['y_true', 'segment_id']
dict_df = {}

for col in columns_names:
    if col.split('_')[0] in ['mha']:
        key_1, key_2, key_3 = col.split('_')[0], col.split('_')[1], '_'.join([col.split('_')[2], col.split('_')[3]])
        dict_df[col] = dict_predictions[key_1][int(key_2)][key_3]
    elif col.split('_')[0] in ['big', 'tiny']:
        key_1, key_2, key_3 = col.split('_')[0], col.split('_')[1], col.split('_')[2]
        dict_df[col] = dict_predictions[key_1][int(key_2)][key_3]
    elif  col.split('_')[1] in ['stft']:
        key_1, key_2, key_3, key_4 = col.split('_')[0], col.split('_')[1], col.split('_')[2], col.split('_')[3]
        dict_df[col] = dict_predictions[f'{key_1}_{key_2}'][int(key_3)][key_4]
    else:
        dict_df[col] = np.concatenate(list(dict_predictions[col].values()))
# print('='*50)       
# for key in dict_df:
#     print(f'{key}: {len(dict_df[key])}')

dict_build_df = {
    'segment_id' : dict_df['segment_id'].squeeze(),
    'mha_pred_mean' : np.concatenate([dict_df['mha_0_pred_mean'], dict_df['mha_1_pred_mean'], 
                                 dict_df['mha_2_pred_mean'], dict_df['mha_3_pred_mean'], dict_df['mha_4_pred_mean']]).squeeze(),
    'mha_pred_std' : np.concatenate([dict_df['mha_0_pred_std'], dict_df['mha_1_pred_std'], 
                                 dict_df['mha_2_pred_std'], dict_df['mha_3_pred_std'], dict_df['mha_4_pred_std']]).squeeze(),
    'mha_conf_mean' : np.concatenate([dict_df['mha_0_conf_mean'], dict_df['mha_1_conf_mean'], 
                                 dict_df['mha_2_conf_mean'], dict_df['mha_3_conf_mean'], dict_df['mha_4_conf_mean']]).squeeze(),
    'mha_conf_std' : np.concatenate([dict_df['mha_0_conf_std'], dict_df['mha_1_conf_std'], 
                                 dict_df['mha_2_conf_std'], dict_df['mha_3_conf_std'], dict_df['mha_4_conf_std']]).squeeze(),
    'cnn_tiny_pred' : np.concatenate([dict_df['tiny_0_pred'], dict_df['tiny_1_pred'], 
                                      dict_df['tiny_2_pred'], dict_df['tiny_3_pred'], dict_df['tiny_4_pred']]).squeeze(),
    'cnn_tiny_conf' : np.concatenate([dict_df['tiny_0_conf'], dict_df['tiny_1_conf'], 
                                      dict_df['tiny_2_conf'], dict_df['tiny_3_conf'], dict_df['tiny_4_conf']]).squeeze(),
    'cnn_big_pred' : np.concatenate([dict_df['big_0_pred'], dict_df['big_1_pred'], 
                                     dict_df['big_2_pred'], dict_df['big_3_pred'], dict_df['big_4_pred']]).squeeze(),
    'cnn_big_conf' : np.concatenate([dict_df['big_0_conf'], dict_df['big_1_conf'], 
                                     dict_df['big_2_conf'], dict_df['big_3_conf'], dict_df['big_4_conf']]).squeeze(),
    'cnn_stft_pred' : np.concatenate([dict_df['cnn2d_stft_0_pred'], dict_df['cnn2d_stft_1_pred'], 
                                     dict_df['cnn2d_stft_2_pred'], dict_df['cnn2d_stft_3_pred'], dict_df['cnn2d_stft_4_pred']]).squeeze(),
    'cnn_stft_conf' : np.concatenate([dict_df['cnn2d_stft_0_conf'], dict_df['cnn2d_stft_1_conf'], 
                                     dict_df['cnn2d_stft_2_conf'], dict_df['cnn2d_stft_3_conf'], dict_df['cnn2d_stft_4_conf']]).squeeze(),
    'lgbm_stft_pred' : np.concatenate([dict_df['lgbm_stft_0_pred'], dict_df['lgbm_stft_1_pred'], 
                                     dict_df['lgbm_stft_2_pred'], dict_df['lgbm_stft_3_pred'], dict_df['lgbm_stft_4_pred']]).squeeze(),
    'lgbm_stft_conf' : np.concatenate([dict_df['lgbm_stft_0_conf'], dict_df['lgbm_stft_1_conf'], 
                                     dict_df['lgbm_stft_2_conf'], dict_df['lgbm_stft_3_conf'], dict_df['lgbm_stft_4_conf']]).squeeze(),
    'y_true' : dict_df['y_true'].squeeze() 
}

df_train_l2 = pd.DataFrame(dict_build_df)
df_train_l2.to_csv('./df_train_l2.csv', index=False)

# del dict_predictions, dict_build_df
# gc.collect()


#############################################################

mha
cnn2d_stft
lgbm_stft



tiny
big
mha
cnn2d_stft
lgbm_stft



tiny
big
mha
cnn2d_stft
lgbm_stft



tiny
big
mha
cnn2d_stft
lgbm_stft



tiny
big
mha
cnn2d_stft
lgbm_stft



tiny
big



KeyError: 'pred'

In [18]:
#############################################################
# 7. Add other models predictions

import pandas as pd

df_train_l2 = pd.read_csv('./df_train_l2.csv')
df_tabular_time = pd.read_csv('./df_time_tabular_validations.csv')
df_tabular_stft2 = pd.read_csv('./df_stft_tabular_validations.csv')

df_tabular_time['time_to_eruption'] = df_tabular_time['time_to_eruption'] /(10**6)
df_tabular_stft2['time_to_eruption'] = df_tabular_stft2['time_to_eruption'] /(10**6)

df_tabular_time.columns = ['segment_id', 'lgbm_time_pred_mean']
df_tabular_stft2.columns = ['segment_id', 'lgbm_stft_pred_mean']

df_added = pd.merge(df_tabular_time, df_tabular_stft2, how='inner', on=['segment_id'])
print(df_added.shape[0], df_tabular_time.shape[0], df_tabular_stft2.shape[0])
df_all = pd.merge(df_train_l2, df_added, how='inner', on=['segment_id'])
print(df_all.shape[0], df_added.shape[0], df_train_l2.shape[0])
df_all.to_csv('./df_all_train_l2.csv', index=False)

#############################################################

4431 4431 4431
4431 4431 4431


In [34]:
# columns_names = []
# for key in dict_predictions.keys():
#     if key in ['mha', 'big', 'tiny', 'cnn2d_stft', 'lgbm_stft']:
#         for fold in list(dict_predictions[key].keys()):
#             if isinstance(dict_predictions[key][fold], dict):
#                 for var in dict_predictions[key][fold]:
#                     columns_names.append(f'{key}_{fold}_{var}')  
#             else:
#                 columns_names.append(f'{key}_{fold}')
                
# columns_names = columns_names + ['y_true', 'segment_id']
# dict_df = {}
# for col in columns_names:
#     if col.split('_')[0] in ['mha', 'big', 'tiny']:
#         key_1, key_2, key_3 = col.split('_')[0], col.split('_')[1], '_'.join([col.split('_')[2], col.split('_')[3]])
#         dict_df[col] = dict_predictions[key_1][int(key_2)][key_3]
#     elif col.split('_')[1] in ['stft'] and col.split('_')[0] != 'lgbm':
#         key_1, key_2, key_3, key_4 = col.split('_')[0], col.split('_')[1], col.split('_')[2], '_'.join(col.split('_')[-2:])
#         dict_df[col] = dict_predictions[f'{key_1}_{key_2}'][int(key_3)][key_4]
#     elif col.split('_')[1] in ['stft'] and col.split('_')[0] == 'lgbm':
#         key_1, key_2, key_3, key_4 = col.split('_')[0], col.split('_')[1], col.split('_')[2], col.split('_')[3]
#         dict_df[col] = dict_predictions[f'{key_1}_{key_2}'][int(key_3)][key_4]
#     else:
#         dict_df[col] = np.concatenate(list(dict_predictions[col].values()))
# # print('='*50)       
# # for key in dict_df:
# #     print(f'{key}: {len(dict_df[key])}')

# dict_build_df = {
#     'segment_id' : dict_df['segment_id'].squeeze(),
#     'mha_pred_mean' : np.concatenate([dict_df['mha_0_pred_mean'], dict_df['mha_1_pred_mean'], 
#                                  dict_df['mha_2_pred_mean'], dict_df['mha_3_pred_mean'], dict_df['mha_4_pred_mean']]).squeeze(),
#     'mha_pred_std' : np.concatenate([dict_df['mha_0_pred_std'], dict_df['mha_1_pred_std'], 
#                                  dict_df['mha_2_pred_std'], dict_df['mha_3_pred_std'], dict_df['mha_4_pred_std']]).squeeze(),
#     'mha_conf_mean' : np.concatenate([dict_df['mha_0_conf_mean'], dict_df['mha_1_conf_mean'], 
#                                  dict_df['mha_2_conf_mean'], dict_df['mha_3_conf_mean'], dict_df['mha_4_conf_mean']]).squeeze(),
#     'mha_conf_std' : np.concatenate([dict_df['mha_0_conf_std'], dict_df['mha_1_conf_std'], 
#                                  dict_df['mha_2_conf_std'], dict_df['mha_3_conf_std'], dict_df['mha_4_conf_std']]).squeeze(),
#     'cnn_tiny_pred_mean' : np.concatenate([dict_df['tiny_0_pred_mean'], dict_df['tiny_1_pred_mean'], 
#                                           dict_df['tiny_2_pred_mean'], dict_df['tiny_3_pred_mean'], dict_df['tiny_4_pred_mean']]).squeeze(),
#     'cnn_tiny_pred_std' : np.concatenate([dict_df['tiny_0_pred_std'], dict_df['tiny_1_pred_std'], 
#                                           dict_df['tiny_2_pred_std'], dict_df['tiny_3_pred_std'], dict_df['tiny_4_pred_std']]).squeeze(),
#     'cnn_tiny_conf_mean' : np.concatenate([dict_df['tiny_0_conf_mean'], dict_df['tiny_1_conf_mean'], 
#                                       dict_df['tiny_2_conf_mean'], dict_df['tiny_3_conf_mean'], dict_df['tiny_4_conf_mean']]).squeeze(),
#     'cnn_tiny_conf_std' : np.concatenate([dict_df['tiny_0_conf_std'], dict_df['tiny_1_conf_std'], 
#                                       dict_df['tiny_2_conf_std'], dict_df['tiny_3_conf_std'], dict_df['tiny_4_conf_std']]).squeeze(),
#     'cnn_big_pred_mean' : np.concatenate([dict_df['big_0_pred_mean'], dict_df['big_1_pred_mean'], 
#                                      dict_df['big_2_pred_mean'], dict_df['big_3_pred_mean'], dict_df['big_4_pred_mean']]).squeeze(),
#     'cnn_big_pred_std' : np.concatenate([dict_df['big_0_pred_std'], dict_df['big_1_pred_std'], 
#                                      dict_df['big_2_pred_std'], dict_df['big_3_pred_std'], dict_df['big_4_pred_std']]).squeeze(),
#     'cnn_big_conf_mean' : np.concatenate([dict_df['big_0_conf_mean'], dict_df['big_1_conf_mean'], 
#                                      dict_df['big_2_conf_mean'], dict_df['big_3_conf_mean'], dict_df['big_4_conf_mean']]).squeeze(),
#     'cnn_big_conf_std' : np.concatenate([dict_df['big_0_conf_std'], dict_df['big_1_conf_std'], 
#                                      dict_df['big_2_conf_std'], dict_df['big_3_conf_std'], dict_df['big_4_conf_std']]).squeeze(),
#     'cnn_stft_pred_mean' : np.concatenate([dict_df['cnn2d_stft_0_pred_mean'], dict_df['cnn2d_stft_1_pred_mean'], 
#                                      dict_df['cnn2d_stft_2_pred_mean'], dict_df['cnn2d_stft_3_pred_mean'], dict_df['cnn2d_stft_4_pred_mean']]).squeeze(),
#     'cnn_stft_pred_std' : np.concatenate([dict_df['cnn2d_stft_0_pred_std'], dict_df['cnn2d_stft_1_pred_std'], 
#                                      dict_df['cnn2d_stft_2_pred_std'], dict_df['cnn2d_stft_3_pred_std'], dict_df['cnn2d_stft_4_pred_std']]).squeeze(),
#     'cnn_stft_conf_mean' : np.concatenate([dict_df['cnn2d_stft_0_conf_mean'], dict_df['cnn2d_stft_1_conf_mean'], 
#                                      dict_df['cnn2d_stft_2_conf_mean'], dict_df['cnn2d_stft_3_conf_mean'], dict_df['cnn2d_stft_4_conf_mean']]).squeeze(),
#     'cnn_stft_conf_std' : np.concatenate([dict_df['cnn2d_stft_0_conf_std'], dict_df['cnn2d_stft_1_conf_std'], 
#                                      dict_df['cnn2d_stft_2_conf_std'], dict_df['cnn2d_stft_3_conf_std'], dict_df['cnn2d_stft_4_conf_std']]).squeeze(),
#     'lgbm_stft_pred' : np.concatenate([dict_df['lgbm_stft_0_pred'], dict_df['lgbm_stft_1_pred'], 
#                                      dict_df['lgbm_stft_2_pred'], dict_df['lgbm_stft_3_pred'], dict_df['lgbm_stft_4_pred']]).squeeze(),
#     'lgbm_stft_conf' : np.concatenate([dict_df['lgbm_stft_0_conf'], dict_df['lgbm_stft_1_conf'], 
#                                      dict_df['lgbm_stft_2_conf'], dict_df['lgbm_stft_3_conf'], dict_df['lgbm_stft_4_conf']]).squeeze(),
#     'y_true' : dict_df['y_true'].squeeze() 
# }

# df_train_l2 = pd.DataFrame(dict_build_df)
# df_train_l2.to_csv('./df_train_l2.csv', index=False)